In [1]:

%matplotlib widget
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os
import sys
from datetime import date
import json
import numpy as np
#print(os.getcwd())

In [2]:
ET_DATA_PATH = 'D:\_MasterArbeit_EEDA\EyeTracking'
SCENE_EVENT_DATA_PATH = 'D:\_MasterArbeit_EEDA\SceneData'
STEERING_DATA_PATH = 'D:\_MasterArbeit_EEDA\Input'
PARTICIPANT_DATA_PATH = 'D:\_MasterArbeit_EEDA\ParticipantCalibrationData'
PROCESSED_DATA_PATH = '..\csv_et'
COMBINED_DATA_PATH = '..\et_comb'
EVENT_DURATION_DATA_PATH = '..\Event_dur'
PLOT_PATH = '..\PLOTS\data_quality'
os.makedirs(os.path.dirname(PROCESSED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(COMBINED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(PLOT_PATH), exist_ok=True)

### Filter EyeValidationError under 1.5

In [3]:
df = pd.DataFrame()

files = glob.glob(f'{PARTICIPANT_DATA_PATH}/*.txt')

for fi in files[:]: # reading all files
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
        tempdf0 = pd.DataFrame.from_dict(tmp,orient='index').T        
        tempdf0[["EyeValidationError.x","EyeValidationError.y","EyeValidationError.z"]] = pd.DataFrame(tempdf0["EyeValidationError"].to_list(), columns=['x', 'y','z'])
        if(tempdf0["EyeValidationError.x"][0]<1.5) and (tempdf0["EyeValidationError.y"][0]<1.5):
            df = pd.concat([df, tempdf0], ignore_index=True)
        
df = df.drop(['IPAddress', 'VRmode',
       'SteeringInputDevice', 'EyeValidationError', 'SeatCalibrationOffset',
       'TrainingDuration', 'ExperimentDuration', 'ApplicationDuration',
       'AverageExperimentFPS', 'SpecialNotes', 'EyeValidationError.z'], axis=1)
display(df)

,ParticipantUuid,ExperimentalCondition,EyeValidationError.x,EyeValidationError.y
0,489749a7e3394cdda22f5f2717a340a8,ManualDriving,0.516909,0.679336
1,8f7015b4bb3b4ffebb8cb4661ec2c690,SemiAutonomous,1.018964,1.401548
2,961c6c338d254e399194523941252667,ManualDriving,0.459474,0.496243
3,bd0b8eecd92c4265b54a63b6cbf4c723,SemiAutonomous,0.932216,0.516343
4,c9e5035541ef43249ec7c78ba02f9977,SemiAutonomous,0.517892,0.699764
5,dcad436d55224cafb236568d2d6ecbf0,SemiAutonomous,0.692415,0.951087


### Check if Incomplete

In [4]:
files = glob.glob(f'{ET_DATA_PATH}/*.txt')
des_index = []
for i in range(0,df['ParticipantUuid'].count()):
    for fi in files[:]: # reading all files
        #print (df['ParticipantUuid'][i])
        if '_Incomplete' in fi:
            if df['ParticipantUuid'][i] in fi:
                if os.path.getsize(fi) < 10 : 
                    des_index.append(i)
                    
                
df = df.iloc[des_index] 
df = df.reset_index(drop=True)
display(df)

,ParticipantUuid,ExperimentalCondition,EyeValidationError.x,EyeValidationError.y
0,489749a7e3394cdda22f5f2717a340a8,ManualDriving,0.516909,0.679336
1,8f7015b4bb3b4ffebb8cb4661ec2c690,SemiAutonomous,1.018964,1.401548
2,bd0b8eecd92c4265b54a63b6cbf4c723,SemiAutonomous,0.932216,0.516343
3,c9e5035541ef43249ec7c78ba02f9977,SemiAutonomous,0.517892,0.699764
4,dcad436d55224cafb236568d2d6ecbf0,SemiAutonomous,0.692415,0.951087


### Load Eyetracking Data

In [5]:
files = glob.glob(f'{ET_DATA_PATH}/*.txt')
for i in range(0,df['ParticipantUuid'].count()):
    et_df = pd.DataFrame()
    for fi in files[:]: # reading all files
        if df['ParticipantUuid'][i] in fi:
            with open(fi, 'r') as j:
                tmp = json.loads(j.read())
            for nS, sample in enumerate(tmp):
                tmpdf = pd.json_normalize(data=sample, sep='_')
                tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
                tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
                tmpdf = tmpdf.drop(['FPS','pupilDiameterMillimetersLeft', 'pupilDiameterMillimetersRight',
       'EyeBlinkingWorldLeftTobii', 'EyeBliningWorldRightTobii',
       'EyeBlinkingLocalLeftTobii', 'EyeBlinkingLocalRightTobii', 'hitObjects',
       'HmdRotation_x', 'HmdRotation_y',
       'HmdRotation_z', 'HmdRotation_w','EyePositionLeftWorld_x',
       'EyePositionLeftWorld_y', 'EyePositionLeftWorld_z',
       'EyeDirectionLeftWorld_x', 'EyeDirectionLeftWorld_y',
       'EyeDirectionLeftWorld_z', 'EyePositionLeftLocal_x',
       'EyePositionLeftLocal_y', 'EyePositionLeftLocal_z',
       'EyeDirectionLeftLocal_x', 'EyeDirectionLeftLocal_y',
       'EyeDirectionLeftLocal_z', 'EyePositionRightWorld_x',
       'EyePositionRightWorld_y', 'EyePositionRightWorld_z',
       'EyePositionRightLocal_x', 'EyePositionRightLocal_y',
       'EyePositionRightLocal_z', 'EyeDirectionRightWorld_x',
       'EyeDirectionRightWorld_y', 'EyeDirectionRightWorld_z',
       'EyeDirectionRightLocal_x', 'EyeDirectionRightLocal_y',
       'EyeDirectionRightLocal_z', 'EyePositionWorldCombinedTobii_x',
       'EyePositionWorldCombinedTobii_y', 'EyePositionWorldCombinedTobii_z',
       'EyePositionLocalCombinedTobii_x', 'EyePositionLocalCombinedTobii_y',
       'EyePositionLocalCombinedTobii_z', 'EyeDirectionWorldCombinedTobii_x',
       'EyeDirectionWorldCombinedTobii_y', 'EyeDirectionWorldCombinedTobii_z',
       'EyeDirectionLocalCombinedTobii_x', 'EyeDirectionLocalCombinedTobii_y',
       'EyeDirectionLocalCombinedTobii_z'],axis=1)
                
                et_df = pd.concat([et_df, tmpdf], ignore_index=True)
                   
    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['timestamp'] = pd.to_datetime(et_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        path = os.path.join(PROCESSED_DATA_PATH,df['ParticipantUuid'][i]+'.csv')
        print(100/df['ParticipantUuid'].count()*(i+1),"%")
        et_df.set_index('timestamp', inplace=True)
        et_df['time_from_start'] = (et_df
#                                    .reset_index()
#                                   .groupby(['uid','scene',], as_index=False)
#                                    .timestamp
                                   .transform(lambda x: 
                                              (x.index - x.index[0])/np.timedelta64(1,'s'))
                                   .UnixTimeStamp
                                   )
        et_df = et_df.reset_index()
        et_df[['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']]=et_df[['EyeDirectionCombinedLocal_x', 'EyeDirectionCombinedLocal_y', 'EyeDirectionCombinedLocal_z']]
        et_df['is_blink'] = ((et_df['EyeOpennessLeftSranipal'] < 0.1)&(et_df['EyeOpennessRightSranipal'] < 0.1))
        et_df.loc[(et_df['is_blink'] == True),['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']] = np.nan
        
        tmpdf = (et_df
                 .groupby(['uid', 'scene'])['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']
                 .transform(lambda x: 
                 x.interpolate(method='polynomial', order=3, limit_direction ='both'))
                 .reset_index()
                )
        et_df[['combinedEyeDir_x_intp', 'combinedEyeDir_y_intp', 'combinedEyeDir_z_intp']] = tmpdf[['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']]


        path = os.path.join(PROCESSED_DATA_PATH,df['ParticipantUuid'][i]+'.csv')
        print(100/df['ParticipantUuid'].count()*(i+1),"%")
        et_df.to_csv(path)
        #display(et_df)

20.0 %
20.0 %


C:\Users\MAXPIN~1\AppData\Local\Temp/ipykernel_13328/3172701261.py:59: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tmpdf = (et_df


40.0 %
40.0 %
60.0 %
60.0 %
80.0 %
80.0 %
100.0 %
100.0 %


### Steeringwheel Data read and write

In [6]:
files = glob.glob(f'{STEERING_DATA_PATH}/*.txt')
for i in range(0,df['ParticipantUuid'].count()):
    et_df = pd.DataFrame()
    for fi in files[:]: # reading all files
        if df['ParticipantUuid'][i] in fi:
            #print("file found ")
            with open(fi, 'r') as j:
                tmp = json.loads(j.read())
            for nS, sample in enumerate(tmp):
                tmpdf = pd.json_normalize(data=sample, sep='_')
                tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
                tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
                et_df = pd.concat([et_df, tmpdf], ignore_index=True)
                #print(sample)
    
    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['timestamp'] = pd.to_datetime(et_df.TimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        #et_df = et_df.rename(columns={"TimeStamp": "UnixTimeStamp"})
        et_df = et_df.reset_index(drop=True)
        path = os.path.join(PROCESSED_DATA_PATH,df['ParticipantUuid'][i]+'_st_df'+'.csv')
        print(100/df['ParticipantUuid'].count()*(i+1),"%")
        et_df.to_csv(path)

display(et_df)

20.0 %
40.0 %
60.0 %
80.0 %
100.0 %


,TimeStamp,ReceivedInput,SteeringInput,AcellerationInput,BrakeInput,uid,scene,timestamp
0,1.635953e+09,True,0.015205,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,TrainingScene,2021-11-03 15:18:21.337622016+00:00
1,1.635953e+09,True,-0.008729,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,TrainingScene,2021-11-03 15:18:24.377266432+00:00
2,1.635953e+09,True,-0.008240,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,TrainingScene,2021-11-03 15:18:24.388237312+00:00
3,1.635953e+09,True,-0.006865,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,TrainingScene,2021-11-03 15:18:24.406189568+00:00
4,1.635953e+09,True,-0.005949,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,TrainingScene,2021-11-03 15:18:24.428130560+00:00
...,...,...,...,...,...,...,...,...
36486,1.635953e+09,True,0.020276,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,CountryRoad,2021-11-03 15:29:40.038520576+00:00
36487,1.635953e+09,True,0.020276,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,CountryRoad,2021-11-03 15:29:40.049491200+00:00
36488,1.635953e+09,True,0.020276,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,CountryRoad,2021-11-03 15:29:40.073427200+00:00
36489,1.635953e+09,True,0.020276,0.0,0.0,dcad436d55224cafb236568d2d6ecbf0,CountryRoad,2021-11-03 15:29:40.083400704+00:00


### Combine Steering and ET

In [7]:
files = glob.glob(f'{PROCESSED_DATA_PATH}\*.csv')
for i in range(0,df['ParticipantUuid'].count()):
    et_df = pd.DataFrame()
    et_df2 = pd.DataFrame()
    ne = pd.DataFrame()
    for fi in files[:]: # reading all files
        if df['ParticipantUuid'][i] in fi:
            if et_df.empty:
                et_df = pd.read_csv(fi)
            else: 
                et_df2 = pd.read_csv(fi)
                                
                et_df2.set_index('timestamp', inplace=True)
                et_df2 = et_df2.reset_index()
                
                #Check if DF are equally long
                b = et_df2.shape[0]- et_df.shape[0]
                et_df2.drop(et_df2.tail(b).index,inplace=True)
                
                et_df['Equal'] = np.where(et_df['timestamp'] == et_df2['timestamp'],'1','0')
                et_df[['SteeringInput', 'AcellerationInput', 'BrakeInput']] = et_df2[['SteeringInput', 'AcellerationInput', 'BrakeInput']]
                #print(ne)
                path = os.path.join(COMBINED_DATA_PATH,df['ParticipantUuid'][i]+'_comb'+'.csv')
                print(100/df['ParticipantUuid'].count()*(i+1),"%")
                et_df.to_csv(path)

                
                        
                #display(et_df)


20.0 %
40.0 %
60.0 %
80.0 %
100.0 %


### Event Loading and Saving

In [8]:
interims_df = pd.DataFrame()
Event_dur_df = pd.DataFrame()

files = glob.glob(f'{SCENE_EVENT_DATA_PATH}/*.txt')

for i in range(0,df['ParticipantUuid'].count()):
    event_samples_df = pd.DataFrame()
    for fi in files[:]: # reading all files
        if df['ParticipantUuid'][i] in fi:
            with open(fi, 'r') as j:
                tmp = json.loads(j.read())
                x = tmp.get('EventBehavior')
                for p in range(len(x)):
                    tmpdf = pd.json_normalize(data=x[p], sep='_')
                    interims_df = tmpdf[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
               'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
                    Event_dur_df = pd.concat([Event_dur_df,interims_df],ignore_index = True)
                    tmpdf2 = pd.json_normalize(tmpdf["dynamicObjectData"])
                    for nS, sample in enumerate(tmpdf2):
                        temp = list(tmpdf2[nS])
                        tmpdf3 = pd.json_normalize(data=temp, sep='_')
                        tmpdf3['uid'] = fi.split('\\')[-1].split('_')[0]
                        tmpdf3['scene'] = fi.split('_')[-1].split('.')[0]
                        tmpdf3[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
               'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']] = interims_df[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
               'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
                        event_samples_df = pd.concat([event_samples_df, tmpdf3], ignore_index=True)
    if not event_samples_df.empty:
        event_samples_df = event_samples_df.drop_duplicates()
        path = os.path.join(EVENT_DURATION_DATA_PATH,df['ParticipantUuid'][i]+'_event_dur'+'.csv')
        event_samples_df['timestamp'] = pd.to_datetime(event_samples_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df['StartofEventTime'] = pd.to_datetime(event_samples_df.StartofEventTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df['EndofEventTime'] = pd.to_datetime(event_samples_df.EndOfEventTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df = event_samples_df.sort_values(by=['timestamp'])
        #event_samples_df = event_samples_df.reset_index(drop=True)
        event_samples_df = event_samples_df.drop(['UnixTimeStamp', 'ObjectName0', 'ObjectName1', 'ObjectName2',
       'ObjectPosition0.x', 'ObjectPosition0.y', 'ObjectPosition0.z','timestamp',
       'ObjectRotation0.x', 'ObjectRotation0.y', 'ObjectRotation0.z',
       'ObjectRotation0.w', 'ObjectPosition1.x', 'ObjectPosition1.y',
       'ObjectPosition1.z', 'ObjectRotation1.x', 'ObjectRotation1.y',
       'ObjectRotation1.z', 'ObjectRotation1.w', 'ObjectPosition2.x',
       'ObjectPosition2.y', 'ObjectPosition2.z', 'ObjectRotation2.x',
       'ObjectRotation2.y', 'ObjectRotation2.z', 'ObjectRotation2.w',  'StartofEventTimeStamp', 'EndOfEventTimeStamp',
        'HitObjectName'
       ],axis=1)
        
        event_samples_df = event_samples_df.drop_duplicates().reset_index(drop=True)
        event_samples_df.to_csv(path)
        
        print(100/df['ParticipantUuid'].count()*(i+1),"%")
        
#display(event_samples_df.columns)

20.0 %
40.0 %
60.0 %
80.0 %
100.0 %


In [9]:
files = glob.glob(f'{EVENT_DURATION_DATA_PATH}\*.csv')
files_comb = glob.glob(f'{COMBINED_DATA_PATH}\*.csv')
#display(event_samples_df)
et_dur_df = pd.DataFrame()
for fi in files_comb[:]:
    et_df = pd.read_csv(fi)
    name = fi.split('\\')[-1].split('_')[0]
    for fi2 in files[:]:
        if name in fi2:
            et_dur_df = pd.read_csv(fi2)
            Start = et_df.iloc[0]['timestamp']
            
            #display(et_dur_df)
            for t in range(0,12):
                et_dur_df['StartofEventTime'][t]=(pd.to_datetime(et_dur_df['StartofEventTime'][t])-pd.to_datetime(Start)).total_seconds()
                et_dur_df['EndofEventTime'][t]=(pd.to_datetime(et_dur_df['EndofEventTime'][t])-pd.to_datetime(Start)).total_seconds()
            path = os.path.join(EVENT_DURATION_DATA_PATH,name+'_event_dur'+'.csv')
            et_dur_df.to_csv(path)
            #display(et_dur_df)

C:\Users\MAXPIN~1\AppData\Local\Temp/ipykernel_13328/1558887823.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  et_dur_df['StartofEventTime'][t]=(pd.to_datetime(et_dur_df['StartofEventTime'][t])-pd.to_datetime(Start)).total_seconds()
C:\Users\MAXPIN~1\AppData\Local\Temp/ipykernel_13328/1558887823.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  et_dur_df['EndofEventTime'][t]=(pd.to_datetime(et_dur_df['EndofEventTime'][t])-pd.to_datetime(Start)).total_seconds()


## Theta Calculations and stuff

In [10]:

files_comb = glob.glob(f'{COMBINED_DATA_PATH}\*.csv')
et_samples_df = pd.DataFrame()
for fi in files_comb[:]:
    et_samples_df = pd.read_csv(fi)
    
    name = fi.split('\\')[-1].split('_')[0]
    et_samples_df.reset_index(inplace=False)
    et_samples_df['eye_theta_h'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_x'], 
                                           et_samples_df['EyeDirectionCombinedLocal_z'])
    et_samples_df['eye_theta_v'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_y'], 
                                           et_samples_df['EyeDirectionCombinedLocal_z'])
    et_samples_df['eye_theta_h_int'] = np.arctan2(et_samples_df['combinedEyeDir_x_intp'], 
                                           et_samples_df['combinedEyeDir_z_intp'])
    et_samples_df['eye_theta_v_int'] = np.arctan2(et_samples_df['combinedEyeDir_y_intp'], 
                                           et_samples_df['combinedEyeDir_z_intp'])
    display(et_samples_df)
    path = os.path.join(COMBINED_DATA_PATH,name+'_comb'+'.csv')
    et_samples_df.to_csv(path)
    print("printed")


,Unnamed: 0,Unnamed: 0.1,timestamp,UnixTimeStamp,TobiiTimeStamp,EyeOpennessLeftSranipal,EyeOpennessRightSranipal,HmdPosition_x,HmdPosition_y,HmdPosition_z,...,combinedEyeDir_y_intp,combinedEyeDir_z_intp,Equal,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h,eye_theta_v,eye_theta_h_int,eye_theta_v_int
0,0,0,2021-11-06 13:51:08.977938176+00:00,1.636207e+09,12365.475586,1.000000,1.000000,-273.177094,386.396790,-496.596313,...,-0.003540,0.999298,0,-0.017985,0.00000,0.0,-0.036905,-0.003543,-0.036905,-0.003543
1,1,1,2021-11-06 13:51:12.017423872+00:00,1.636207e+09,12375.961914,1.000000,1.000000,-273.177490,386.396179,-496.594696,...,-0.087891,0.992645,1,-0.001305,0.00000,0.0,0.083505,-0.088312,0.083505,-0.088312
2,2,2,2021-11-06 13:51:12.028014592+00:00,1.636207e+09,12375.978516,1.000000,1.000000,-273.178711,386.383575,-496.574463,...,-0.108978,0.992722,1,-0.001305,0.00000,0.0,0.051431,-0.109339,0.051431,-0.109339
3,3,3,2021-11-06 13:51:12.047162368+00:00,1.636207e+09,12375.998047,1.000000,1.000000,-273.177094,386.383362,-496.573334,...,-0.080017,0.995850,1,-0.001305,0.00000,0.0,0.043396,-0.080178,0.043396,-0.080178
4,4,4,2021-11-06 13:51:12.068107008+00:00,1.636207e+09,12376.018555,1.000000,1.000000,-273.175476,386.383148,-496.572327,...,-0.097916,0.994705,1,-0.001305,0.00000,0.0,0.030824,-0.098121,0.030824,-0.098121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48732,48732,48732,2021-11-06 14:06:09.564778240+00:00,1.636208e+09,13273.514648,0.850338,0.915034,-1499.260986,131.253998,2380.560791,...,-0.025070,0.996582,0,-0.010592,0.33401,0.0,-0.078674,-0.025151,-0.078674,-0.025151
48733,48733,48733,2021-11-06 14:06:09.574750464+00:00,1.636208e+09,13273.526367,0.850338,0.915034,-1498.994995,131.230194,2380.860840,...,-0.025070,0.996582,0,-0.011509,0.33401,0.0,-0.078674,-0.025151,-0.078674,-0.025151
48734,48734,48734,2021-11-06 14:06:09.597690112+00:00,1.636208e+09,13273.547852,0.872943,0.925639,-1498.728027,131.206619,2381.160889,...,-0.036407,0.995895,0,-0.013830,0.33401,0.0,-0.082838,-0.036541,-0.082838,-0.036541
48735,48735,48735,2021-11-06 14:06:09.608660992+00:00,1.636208e+09,13273.559570,0.872943,0.925639,-1498.461060,131.182968,2381.461426,...,-0.036407,0.995895,0,-0.014747,0.33401,0.0,-0.082838,-0.036541,-0.082838,-0.036541


printed


,Unnamed: 0,Unnamed: 0.1,timestamp,UnixTimeStamp,TobiiTimeStamp,EyeOpennessLeftSranipal,EyeOpennessRightSranipal,HmdPosition_x,HmdPosition_y,HmdPosition_z,...,combinedEyeDir_y_intp,combinedEyeDir_z_intp,Equal,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h,eye_theta_v,eye_theta_h_int,eye_theta_v_int
0,0,0,2021-11-06 14:12:46.818085888+00:00,1.636208e+09,0.000000,0.0,0.14151,-273.149384,386.370392,-496.583740,...,-1.000000,-1.000000,0,0.000389,0.0,0.020019,2.356194,-2.356194,2.356194,-2.356194
1,1,1,2021-11-06 14:12:49.823461888+00:00,1.636208e+09,13673.767578,1.0,1.00000,-273.149536,386.370331,-496.583466,...,0.102783,0.994217,1,0.000000,0.0,0.000000,0.031176,0.103015,0.031176,0.103015
2,2,2,2021-11-06 14:12:49.833606400+00:00,1.636208e+09,13673.784180,1.0,1.00000,-273.161163,386.360229,-496.572205,...,0.102783,0.994217,1,0.000000,0.0,0.000000,0.031176,0.103015,0.031176,0.103015
3,3,3,2021-11-06 14:12:49.851117568+00:00,1.636208e+09,13673.801758,1.0,1.00000,-273.161041,386.360809,-496.572174,...,0.098633,0.994186,1,0.000000,0.0,0.000000,0.043285,0.098886,0.043285,0.098886
4,4,4,2021-11-06 14:12:49.864082432+00:00,1.636208e+09,13673.814453,1.0,1.00000,-273.160767,386.361206,-496.571777,...,0.098343,0.994385,1,0.000000,0.0,0.000000,0.038895,0.098578,0.038895,0.098578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34792,34792,34792,2021-11-06 14:23:44.785566720+00:00,1.636209e+09,14328.736328,1.0,1.00000,-1497.644775,131.186768,2380.338135,...,-0.017212,0.987396,1,0.012425,0.0,0.000000,-0.157968,-0.017430,-0.157968,-0.017430
34793,34793,34793,2021-11-06 14:23:44.796537088+00:00,1.636209e+09,14328.748047,1.0,1.00000,-1497.577271,131.180023,2380.429688,...,-0.014816,0.985962,1,0.011967,0.0,0.000000,-0.167085,-0.015026,-0.167085,-0.015026
34794,34794,34794,2021-11-06 14:23:44.818478848+00:00,1.636209e+09,14328.769531,1.0,1.00000,-1497.508667,131.173080,2380.523438,...,-0.016373,0.986298,1,0.011967,0.0,0.000000,-0.164863,-0.016599,-0.164863,-0.016599
34795,34795,34795,2021-11-06 14:23:44.841470208+00:00,1.636209e+09,14328.791992,1.0,1.00000,-1497.439331,131.166275,2380.616699,...,-0.002609,0.996170,1,0.011967,0.0,0.000000,-0.087362,-0.002619,-0.087362,-0.002619


printed


,Unnamed: 0,Unnamed: 0.1,timestamp,UnixTimeStamp,TobiiTimeStamp,EyeOpennessLeftSranipal,EyeOpennessRightSranipal,HmdPosition_x,HmdPosition_y,HmdPosition_z,...,combinedEyeDir_y_intp,combinedEyeDir_z_intp,Equal,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h,eye_theta_v,eye_theta_h_int,eye_theta_v_int
0,0,0,2021-11-04 12:31:50.172208896+00:00,1.636029e+09,9637.833008,1.000000,1.000000,-273.259338,386.386566,-496.570679,...,0.158035,0.986801,0,0.000000,0.0,0.0,0.035426,0.158801,0.035426,0.158801
1,1,1,2021-11-04 12:31:53.244309248+00:00,1.636029e+09,9648.453125,1.000000,1.000000,-273.259460,386.386505,-496.570709,...,0.042648,0.998749,1,0.000000,0.0,0.0,0.026028,0.042676,0.026028,0.042676
2,2,2,2021-11-04 12:31:53.254284032+00:00,1.636029e+09,9648.469727,1.000000,1.000000,-273.260345,386.380554,-496.583160,...,0.033676,0.998764,1,0.000000,0.0,0.0,0.036330,0.033705,0.036330,0.033705
3,3,3,2021-11-04 12:31:53.272235264+00:00,1.636029e+09,9648.488281,1.000000,1.000000,-273.260559,386.381409,-496.583740,...,0.019714,0.998199,1,-0.000389,0.0,0.0,0.056514,0.019747,0.056514,0.019747
4,4,4,2021-11-04 12:31:53.293179136+00:00,1.636029e+09,9648.509766,1.000000,1.000000,-273.260742,386.382233,-496.583893,...,0.018158,0.998184,1,-0.001764,0.0,0.0,0.057429,0.018189,0.057429,0.018189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34389,34389,34389,2021-11-04 12:42:26.590889216+00:00,1.636030e+09,10281.806641,0.023742,0.349810,-1497.594482,131.177292,2380.498291,...,-0.216278,0.975403,0,-0.005002,0.0,0.0,0.043274,-0.218202,0.043274,-0.218202
34390,34390,34390,2021-11-04 12:42:26.612830720+00:00,1.636030e+09,10281.829102,0.060627,0.381543,-1497.525513,131.170120,2380.592285,...,-0.222244,0.973282,0,-0.004544,0.0,0.0,0.058896,-0.224496,0.058896,-0.224496
34391,34391,34391,2021-11-04 12:42:26.635768832+00:00,1.636030e+09,10281.851562,0.000000,0.312392,-1497.456787,131.162781,2380.686279,...,-0.310974,0.948715,0,-0.004544,0.0,0.0,0.059792,-0.316748,0.059792,-0.316748
34392,34392,34392,2021-11-04 12:42:26.657709824+00:00,1.636030e+09,10281.873047,0.000000,0.313668,-1497.385498,131.154938,2380.783691,...,-0.317490,0.947205,0,-0.004544,0.0,0.0,0.046876,-0.323417,0.046876,-0.323417


printed


,Unnamed: 0,Unnamed: 0.1,timestamp,UnixTimeStamp,TobiiTimeStamp,EyeOpennessLeftSranipal,EyeOpennessRightSranipal,HmdPosition_x,HmdPosition_y,HmdPosition_z,...,combinedEyeDir_y_intp,combinedEyeDir_z_intp,Equal,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h,eye_theta_v,eye_theta_h_int,eye_theta_v_int
0,0,0,2021-11-06 13:09:52.430521088+00:00,1.636204e+09,1150.362915,1.0,1.0,-272.961792,386.401276,-496.521332,...,0.027985,0.994492,1,-0.065610,0.0,0.0,-0.101210,0.028132,-0.101210,0.028132
1,1,1,2021-11-06 13:09:55.441755904+00:00,1.636204e+09,1160.598022,1.0,1.0,-272.961884,386.401184,-496.521362,...,0.229691,0.971878,1,0.009645,0.0,0.0,0.053064,0.232078,0.053064,0.232078
2,2,2,2021-11-06 13:09:55.453724160+00:00,1.636204e+09,1160.618286,1.0,1.0,-272.967316,386.401062,-496.514923,...,0.229691,0.971878,1,0.009645,0.0,0.0,0.053064,0.232078,0.053064,0.232078
3,3,3,2021-11-06 13:09:55.476663552+00:00,1.636204e+09,1160.640991,1.0,1.0,-272.966949,386.401581,-496.514832,...,0.156677,0.986984,1,0.009645,0.0,0.0,0.036346,0.157430,0.036346,0.157430
4,4,4,2021-11-06 13:09:55.497606656+00:00,1.636204e+09,1160.661621,1.0,1.0,-272.966461,386.402008,-496.515381,...,0.156265,0.986908,1,0.009645,0.0,0.0,0.040363,0.157035,0.040363,0.157035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35766,35766,35766,2021-11-06 13:20:59.847223808+00:00,1.636205e+09,1825.011963,1.0,1.0,-1497.527100,131.228027,2380.240479,...,0.076035,0.972580,1,0.011936,0.0,0.0,0.222221,0.078020,0.222221,0.078020
35767,35767,35767,2021-11-06 13:20:59.869165056+00:00,1.636205e+09,1825.033813,1.0,1.0,-1497.459839,131.221451,2380.332275,...,0.076172,0.972534,1,0.011936,0.0,0.0,0.222395,0.078164,0.222395,0.078164
35768,35768,35768,2021-11-06 13:20:59.880135424+00:00,1.636205e+09,1825.045166,1.0,1.0,-1497.390625,131.214600,2380.425537,...,0.075195,0.972748,1,0.011936,0.0,0.0,0.221721,0.077149,0.221721,0.077149
35769,35769,35769,2021-11-06 13:20:59.902077440+00:00,1.636205e+09,1825.067261,1.0,1.0,-1497.321411,131.207886,2380.518799,...,0.079590,0.972397,1,0.011936,0.0,0.0,0.221784,0.081667,0.221784,0.081667


printed


,Unnamed: 0,Unnamed: 0.1,timestamp,UnixTimeStamp,TobiiTimeStamp,EyeOpennessLeftSranipal,EyeOpennessRightSranipal,HmdPosition_x,HmdPosition_y,HmdPosition_z,...,combinedEyeDir_y_intp,combinedEyeDir_z_intp,Equal,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h,eye_theta_v,eye_theta_h_int,eye_theta_v_int
0,0,0,2021-11-03 15:18:21.338619904+00:00,1.635953e+09,1016.327209,1.000000,1.000000,-273.225098,386.383728,-496.587494,...,-0.236588,0.971558,0,0.015205,0.0,0.0,-0.010287,-0.238865,-0.010287,-0.238865
1,1,1,2021-11-03 15:18:24.377266432+00:00,1.635953e+09,1026.779053,1.000000,1.000000,-273.225189,386.383759,-496.587616,...,-0.183685,0.980286,1,-0.008729,0.0,0.0,0.073957,-0.185231,0.073957,-0.185231
2,2,2,2021-11-03 15:18:24.388237312+00:00,1.635953e+09,1026.796753,1.000000,1.000000,-273.222656,386.343781,-496.565308,...,-0.182678,0.980667,1,-0.008240,0.0,0.0,0.071236,-0.184169,0.071236,-0.184169
3,3,3,2021-11-03 15:18:24.406189568+00:00,1.635953e+09,1026.815430,1.000000,1.000000,-273.223053,386.343597,-496.565887,...,-0.181305,0.983353,1,-0.006865,0.0,0.0,0.012056,-0.182327,0.012056,-0.182327
4,4,4,2021-11-03 15:18:24.428130560+00:00,1.635953e+09,1026.837036,1.000000,1.000000,-273.223511,386.343567,-496.566315,...,-0.176910,0.984116,1,-0.005949,0.0,0.0,0.014589,-0.177866,0.014589,-0.177866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36486,36486,36486,2021-11-03 15:29:40.038520576+00:00,1.635953e+09,1702.447998,1.000000,1.000000,-1497.598267,131.191254,2380.482178,...,-0.138000,0.989288,1,0.020276,0.0,0.0,0.047793,-0.138600,0.047793,-0.138600
36487,36487,36487,2021-11-03 15:29:40.049491200+00:00,1.635953e+09,1702.459961,1.000000,1.000000,-1497.527222,131.184265,2380.579102,...,-0.100784,0.990479,1,0.020276,0.0,0.0,0.094340,-0.101404,0.094340,-0.101404
36488,36488,36488,2021-11-03 15:29:40.073427200+00:00,1.635953e+09,1702.482300,0.932693,0.747978,-1497.453003,131.177017,2380.679199,...,-0.026794,0.994080,1,0.020276,0.0,0.0,0.105413,-0.026947,0.105413,-0.026947
36489,36489,36489,2021-11-03 15:29:40.083400704+00:00,1.635953e+09,1702.493530,0.932693,0.747978,-1497.378784,131.169754,2380.778564,...,-0.026794,0.994080,1,0.020276,0.0,0.0,0.105413,-0.026947,0.105413,-0.026947


printed


In [11]:
files = glob.glob(f'{EVENT_DURATION_DATA_PATH}\*.csv')
files_comb = glob.glob(f'{COMBINED_DATA_PATH}\*.csv')
#display(event_samples_df)
et_dur_df = pd.DataFrame()
for fi in files_comb[:]:
    et_df = pd.read_csv(fi)
    name = fi.split('\\')[-1].split('_')[0]
    for fi2 in files[:]:
        if name in fi2:
            et_dur_df = pd.read_csv(fi2)
            xposition = et_dur_df['StartofEventTime']
            x2position = et_dur_df['EndofEventTime']
    ax = et_df.plot(kind="scatter", x="time_from_start",y= "eye_theta_h", color="b", label="original",figsize=(30,5))
    et_df.plot(kind="scatter",x="time_from_start",y="eye_theta_h_int", color="r", label="interpolated", ax=ax, alpha=0.09)
    et_df.plot(kind="scatter",x="time_from_start",y="SteeringInput", color="y", label="steering", ax=ax, alpha=0.09)
    et_df.plot(kind="scatter",x="time_from_start",y="Equal", color="k", label="timestamps equal", ax=ax, alpha=0.01)
    
    
    for xc in xposition:
        ax.axvline(x=xc, color='g', linestyle='-',label = 'Event Start')
    for xc2 in x2position:
        ax.axvline(x=xc2, color='k', linestyle='-',label = 'Event End')
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …